In [49]:
import pandas as pd
import numpy as np
import random

import quantlib
import quantlib.indicators_cal as indicators_cal
import quantlib.diagnostics_utils as diagnostics_utils

In [60]:
df = pd.read_excel("../crypto_historical_data.xlsx", engine="openpyxl", index_col='open_time')
df.head()

,BTCUSDT open,BTCUSDT high,BTCUSDT low,BTCUSDT close,BTCUSDT quote_volume,BTCUSDT count,BTCUSDT taker_buy_quote_volume,ETHUSDT open,ETHUSDT high,ETHUSDT low,...,AGIXUSDT avg trade size,AGIXUSDT taker %,AGIXUSDT % ret,AGIXUSDT % ret vol,AGIXUSDT active,GMXUSDT avg trade size,GMXUSDT taker %,GMXUSDT % ret,GMXUSDT % ret vol,GMXUSDT active
open_time,,,,,,,,,,,,,,,,,,,,,
2023-01-07 23:00:00,16930.4,16940.4,16925.6,16936.5,3.019290e+07,11858,1.500107e+07,1262.36,1263.87,1261.00,...,189.588055,0.396481,NaN,NaN,True,220.78668,0.520373,NaN,NaN,True
2023-01-08 00:00:00,16936.5,16949.3,16905.0,16919.4,1.155826e+08,32754,5.906914e+07,1263.64,1264.13,1256.30,...,189.588055,0.396481,0.0,NaN,False,220.78668,0.520373,0.0,NaN,False
2023-01-08 01:00:00,16919.3,16930.0,16908.6,16924.8,6.354241e+07,20364,3.528297e+07,1260.71,1261.87,1258.60,...,189.588055,0.396481,0.0,NaN,False,220.78668,0.520373,0.0,NaN,False
2023-01-08 02:00:00,16924.9,16927.4,16912.3,16925.4,3.022994e+07,12042,1.323348e+07,1261.74,1262.41,1258.77,...,189.588055,0.396481,0.0,NaN,False,220.78668,0.520373,0.0,NaN,False
2023-01-08 03:00:00,16925.5,16939.5,16923.2,16939.0,4.565200e+07,16132,2.718141e+07,1261.25,1262.56,1259.86,...,189.588055,0.396481,0.0,NaN,False,220.78668,0.520373,0.0,NaN,False


In [81]:
def find_target_symbol_df(df, symbol):
    cols = df.columns
    target_cols = [x for x in cols if symbol in x]
    return df[target_cols]

symbol = "OPUSDT"
exp_df = find_target_symbol_df(df, symbol)
exp_df

,OPUSDT open,OPUSDT high,OPUSDT low,OPUSDT close,OPUSDT quote_volume,OPUSDT count,OPUSDT taker_buy_quote_volume,OPUSDT avg trade size,OPUSDT taker %,OPUSDT % ret,OPUSDT % ret vol,OPUSDT active
open_time,,,,,,,,,,,,
2023-01-07 23:00:00,1.0876,1.0954,1.0860,1.0944,2.117867e+06,7774,1.187269e+06,272.429488,0.560596,NaN,NaN,True
2023-01-08 00:00:00,1.0942,1.1000,1.0615,1.0737,7.521097e+06,23237,3.220695e+06,323.669004,0.428221,-0.018914,NaN,True
2023-01-08 01:00:00,1.0738,1.0854,1.0698,1.0832,3.180782e+06,11317,1.625226e+06,281.062290,0.510952,0.008848,NaN,True
2023-01-08 02:00:00,1.0830,1.0863,1.0768,1.0831,1.432330e+06,5784,6.353013e+05,247.636553,0.443544,-0.000092,NaN,True
2023-01-08 03:00:00,1.0830,1.0948,1.0805,1.0922,2.441761e+06,8768,1.440166e+06,278.485491,0.589806,0.008402,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-18 10:00:00,2.7705,2.7752,2.7064,2.7228,1.479287e+07,36359,5.973413e+06,406.855874,0.403803,-0.017217,0.009603,True
2023-02-18 11:00:00,2.7226,2.7447,2.7210,2.7307,9.040629e+06,27201,4.501948e+06,332.363849,0.497968,0.002901,0.009229,True
2023-02-18 12:00:00,2.7309,2.7661,2.7157,2.7562,1.567272e+07,43887,8.052113e+06,357.115419,0.513766,0.009338,0.009179,True


In [82]:
ema_pairs = [(7, 25), (5, 10)]
ema_pair = ema_pairs[0]


for ema in ema_pair:
    exp_df[f"{symbol} {ema} ema"] = indicators_cal.ema_series(exp_df[f"{symbol} close"], n=ema)

exp_df["ema gap"] = exp_df[f"{symbol} {ema_pair[0]} ema"] - exp_df[f"{symbol} {ema_pair[1]} ema"]
exp_df

/var/folders/rf/hq7456ks1lzbgdqbpk9nl69w0000gn/T/ipykernel_31545/3505632821.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df[f"{symbol} {ema} ema"] = indicators_cal.ema_series(exp_df[f"{symbol} close"], n=ema)
/var/folders/rf/hq7456ks1lzbgdqbpk9nl69w0000gn/T/ipykernel_31545/3505632821.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_df[f"{symbol} {ema} ema"] = indicators_cal.ema_series(exp_df[f"{symbol} close"], n=ema)
/var/folders/rf/hq7456ks1lzbgdqbpk9nl69w0000gn/T/ipykernel_31545/3505632

,OPUSDT open,OPUSDT high,OPUSDT low,OPUSDT close,OPUSDT quote_volume,OPUSDT count,OPUSDT taker_buy_quote_volume,OPUSDT avg trade size,OPUSDT taker %,OPUSDT % ret,OPUSDT % ret vol,OPUSDT active,OPUSDT 7 ema,OPUSDT 25 ema,ema gap
open_time,,,,,,,,,,,,,,,
2023-01-07 23:00:00,1.0876,1.0954,1.0860,1.0944,2.117867e+06,7774,1.187269e+06,272.429488,0.560596,NaN,NaN,True,NaN,NaN,NaN
2023-01-08 00:00:00,1.0942,1.1000,1.0615,1.0737,7.521097e+06,23237,3.220695e+06,323.669004,0.428221,-0.018914,NaN,True,NaN,NaN,NaN
2023-01-08 01:00:00,1.0738,1.0854,1.0698,1.0832,3.180782e+06,11317,1.625226e+06,281.062290,0.510952,0.008848,NaN,True,NaN,NaN,NaN
2023-01-08 02:00:00,1.0830,1.0863,1.0768,1.0831,1.432330e+06,5784,6.353013e+05,247.636553,0.443544,-0.000092,NaN,True,NaN,NaN,NaN
2023-01-08 03:00:00,1.0830,1.0948,1.0805,1.0922,2.441761e+06,8768,1.440166e+06,278.485491,0.589806,0.008402,NaN,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-18 10:00:00,2.7705,2.7752,2.7064,2.7228,1.479287e+07,36359,5.973413e+06,406.855874,0.403803,-0.017217,0.009603,True,2.769751,2.779141,-0.009389
2023-02-18 11:00:00,2.7226,2.7447,2.7210,2.7307,9.040629e+06,27201,4.501948e+06,332.363849,0.497968,0.002901,0.009229,True,2.759989,2.775414,-0.015426
2023-02-18 12:00:00,2.7309,2.7661,2.7157,2.7562,1.567272e+07,43887,8.052113e+06,357.115419,0.513766,0.009338,0.009179,True,2.759041,2.773936,-0.014895


In [83]:
portfolio_df = pd.DataFrame(index=exp_df.index).reset_index()
portfolio_df.loc[0, "capital"] = 10000
portfolio_df.loc[0, f"{symbol} units"] = 0
portfolio_df.loc[0, "pnl"] = 0


"""Run simulation"""

for i in portfolio_df.index:
    open_time = portfolio_df.loc[i, "open_time"]
    ema_gap = exp_df.loc[open_time, "ema gap"]
    
    if pd.isna(ema_gap) or i == 0:
        portfolio_df.loc[i, "capital"] = 10000
        portfolio_df.loc[i, f"{symbol} units"] = 0
        portfolio_df.loc[i, "pnl"] = 0
        portfolio_df.loc[i, "capital_ret"] = 0
        continue
    
    time_prev = portfolio_df.loc[i-1, "open_time"]
    prev_position = portfolio_df.loc[i-1, f"{symbol} units"]
    unit_price = exp_df.loc[open_time, f"{symbol} close"]
    """
    Get PnL
    """
    pnl, capital_ret = 0, 0
    if prev_position != 0:
        price_change = unit_price - exp_df.loc[time_prev, f"{symbol} close"]
        pnl = price_change * prev_position
        capital_ret = exp_df.loc[open_time, f"{symbol} % ret"]

    portfolio_df.loc[i, "pnl"] = pnl
    portfolio_df.loc[i, "capital ret"] = capital_ret
    portfolio_df.loc[i, "capital"] = portfolio_df.loc[i-1, "capital"] + pnl
    """
    Get positions
    """
    print (portfolio_df.loc[i, "capital ret"], prev_position, ema_gap)
    if ema_gap > 0:
        # Check the previous ema_gap
        if exp_df.loc[time_prev, "ema gap"] >= 0:
            portfolio_df.loc[i, f"{symbol} units"] = prev_position
        elif exp_df.loc[time_prev, "ema gap"] < 0:
            position = portfolio_df.loc[i, "capital"] / unit_price
            portfolio_df.loc[i, f"{symbol} units"] = position
        else: # Could be nan
            portfolio_df.loc[i, f"{symbol} units"] = prev_position
            
    elif ema_gap <= 0:
        portfolio_df.loc[i, f"{symbol} units"] = 0
    """
    Other calculations if needed
    """

            
                 
    


0.0 0.0 0.07635218456148385
0.0 0.0 0.07891259995957434
0.0 0.0 0.08140033754364517
0.0 0.0 0.07800914937985493
0.0 0.0 0.07551122777838759
0.0 0.0 0.08357975844317189
0.0 0.0 0.0865198997025769
0.0 0.0 0.08490115327250014
0.0 0.0 0.08441507587336039
0.0 0.0 0.08223019390716058
0.0 0.0 0.07882469497080757
0.0 0.0 0.074812720803894
0.0 0.0 0.06644264790364818
0.0 0.0 0.06276254651302304
0.0 0.0 0.056965504271186074
0.0 0.0 0.05569898456008371
0.0 0.0 0.0577837212108574
0.0 0.0 0.05494846733041503
0.0 0.0 0.04886543642606256
0.0 0.0 0.03789900279177871
0.0 0.0 0.026964567914355753
0.0 0.0 0.018477948231632846
0.0 0.0 0.01212609713913948
0.0 0.0 0.00849931374550672
0.0 0.0 0.00513270767019347
0.0 0.0 0.006562889855165288
0.0 0.0 0.0052806527167772455
0.0 0.0 0.0001721682994937357
0.0 0.0 -0.001204324149000513
0.0 0.0 -0.0002995049950926809
0.0 0.0 0.0025653602844415424
-0.01051887903289117 7849.90972603815 0.00218016393403353
-0.00809202697342326 7849.90972603815 0.00010617869271212221
0.

In [84]:
diagnostics_utils.save_backtests(
            portfolio_df=portfolio_df, instruments=symbol, brokerage_used="binance", sysname=f"{symbol}_MA"
            )

/Users/larrycao/Desktop/Multi-Strategy-Quant-System/quantlib/diagnostics_utils.py:15: UserWarning: Ignoring `palette` because no `hue` variable has been assigned.
  ax = sns.lineplot(data=portfolio_df["cum ret"], linewidth=2.5, palette="deep")


In [35]:
exp_df.loc[exp_df["ema gap"]<0]["ema gap"]

open_time
2023-01-05 20:00:00   -0.000396
2023-01-05 21:00:00   -0.000368
2023-01-05 22:00:00   -0.000382
2023-01-05 23:00:00   -0.000531
2023-01-06 00:00:00   -0.000516
                         ...   
2023-02-14 08:00:00   -0.000150
2023-02-14 09:00:00   -0.000065
2023-02-14 10:00:00   -0.000046
2023-02-14 11:00:00   -0.000041
2023-02-14 12:00:00   -0.000027
Name: ema gap, Length: 471, dtype: float64